<a href="https://colab.research.google.com/github/connor9899/DataScience/blob/main/Tabla_presupuesto_diario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tabla presupuesto diario

En este codigo se unifican 3 tablas.

1.   Tabla de presupuesto (df_presupuesto)
2.   Tabla de los meses y dias habiles (df_dias_habiles)
3.   Tabla calendari de los dias habiles del mes (df_fechas)

Para generar la tabla de presupuesto diario (df_final)



# Librerias

In [14]:
import pandas as pd;
from collections import Counter;

#  Conección a google drive

In [8]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Conección al archivo excel


In [15]:
df_dias_habiles = pd.read_excel("/content/gdrive/MyDrive/DataSet/Tabla_Presupuesto_Python.xlsx", sheet_name="Dias_Habiles")

In [17]:
df_fechas = pd.read_excel("/content/gdrive/MyDrive/DataSet/Tabla_Presupuesto_Python.xlsx", sheet_name="Dias_Habiles_Especifico")

In [19]:
df_presupuesto = pd.read_excel("/content/gdrive/MyDrive/DataSet/Tabla_Presupuesto_Python.xlsx", sheet_name="presupuesto_anual")

Visualización de las 5 primeras filas de cada archivo e información del DataFrame

In [16]:
# Ver las primeras 5 filas
print(df_dias_habiles.head())

# Ver información sobre el DataFrame
print(df_dias_habiles.info())

   Orden      Mes  Dias_habiles
0      1    Enero            22
1      2  Febrero            20
2      3    Marzo            21
3      4    Abril            21
4      5     Mayo            20
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Orden         12 non-null     int64 
 1   Mes           12 non-null     object
 2   Dias_habiles  12 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 416.0+ bytes
None


In [18]:
# Ver las primeras 5 filas
print(df_fechas.head())

# Ver información sobre el DataFrame
print(df_fechas.info())

  Fecha_Dias_Habiles  N_Mes    Mes
0         2025-01-02      1  Enero
1         2025-01-03      1  Enero
2         2025-01-06      1  Enero
3         2025-01-07      1  Enero
4         2025-01-08      1  Enero
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Fecha_Dias_Habiles  250 non-null    datetime64[ns]
 1   N_Mes               250 non-null    int64         
 2   Mes                 250 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 6.0+ KB
None


In [20]:
# Ver las primeras 5 filas
print(df_presupuesto.head())

# Ver información sobre el DataFrame
print(df_presupuesto.info())

   Categoria              Area   Sub_Categoria  Presupuesto_Mensual
0  Mayorista  corporativo real        deposito                  188
1  Mayorista  corporativo real          Noneda                   16
2  Minorista  corporativo real       derivados                   39
3  Minorista               NaN          pactos                   50
4  Minorista               NaN  estructturados                   60
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Categoria            5 non-null      object
 1   Area                 3 non-null      object
 2   Sub_Categoria        5 non-null      object
 3   Presupuesto_Mensual  5 non-null      int64 
dtypes: int64(1), object(3)
memory usage: 288.0+ bytes
None


Genera nueva tabla y se realiza la consolidación y campos de los calculos

In [21]:
resultado = []
for _, fecha_row in df_fechas.iterrows():
    # Obtener los días hábiles para el mes actual
    dias_habiles = df_dias_habiles[df_dias_habiles['Mes'] == fecha_row['Mes']]['Dias_habiles'].iloc[0]

    for _, presup_row in df_presupuesto.iterrows():
        # Calcular el presupuesto diario
        presupuesto_diario = presup_row['Presupuesto_Mensual'] / dias_habiles

        nueva_fila = {
            'Fecha_Dias_Habiles': fecha_row['Fecha_Dias_Habiles'],
            'N_Mes': fecha_row['N_Mes'],
            'Mes': fecha_row['Mes'],
            'Segmento de mesa': presup_row['Segmento de mesa'],
            'Mayorista': presup_row['Mayorista'],
            'Corporativas': presup_row['Corporativas'],
            'Sub Mayoristas / Sub Minoristas': presup_row['Sub Mayoristas / Sub Minoristas'],
            'Producto Generico': presup_row['Producto Generico'],
            'PP Base Mensual(MM$)': presup_row['PP Base Mensual(MM$)'],
            'Dias_Habiles_Mes': dias_habiles,
            'Presupuesto_Diario': round(presupuesto_diario, 2)  # Redondeamos a 2 decimales
        }
        resultado.append(nueva_fila)

In [22]:
# Crear DataFrame final
df_final = pd.DataFrame(resultado)

# Ordenar las columnas
df_final = df_final[[
    'Fecha_Dias_Habiles', 'N_Mes', 'Mes',
    'Categoria', 'Area', 'Sub_Categoria',
    'Presupuesto_Mensual', 'Dias_Habiles_Mes', 'Presupuesto_Diario'
]]


In [23]:
df_final.to_excel('resultado_con_presupuesto_diario.xlsx', index=False)

# Mostrar información del resultado
print("Primeras 10 filas del resultado:")
print(df_final.head(10))
print("\nTotal de filas:", len(df_final))

Primeras 10 filas del resultado:
  Fecha_Dias_Habiles  N_Mes    Mes  Categoria              Area  \
0         2025-01-02      1  Enero  Mayorista  corporativo real   
1         2025-01-02      1  Enero  Mayorista  corporativo real   
2         2025-01-02      1  Enero  Minorista  corporativo real   
3         2025-01-02      1  Enero  Minorista               NaN   
4         2025-01-02      1  Enero  Minorista               NaN   
5         2025-01-03      1  Enero  Mayorista  corporativo real   
6         2025-01-03      1  Enero  Mayorista  corporativo real   
7         2025-01-03      1  Enero  Minorista  corporativo real   
8         2025-01-03      1  Enero  Minorista               NaN   
9         2025-01-03      1  Enero  Minorista               NaN   

    Sub_Categoria  Presupuesto_Mensual  Dias_Habiles_Mes  Presupuesto_Diario  
0        deposito                  188                22                8.55  
1          Noneda                   16                22               